<a href="https://colab.research.google.com/github/Pujitha2005/Fruit/blob/main/Fruit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'aids-data-quest-society:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F70769%2F7731816%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240331%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240331T112957Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6c6e2150fc6bce92c1279065b5093fa28f58edb08ee5de302d2c7509d90f8300e31501736d351a7bd9945375ed050c0326faf2a0462a457b9330729593b2dc121e74de28e62d594f140f0b02533a0fba54b57e98c2f8034b0d75943b2df7255668cd552993765c46b627a2074321184f00c66db284277845b73e1ce3bf1105832fd25ff7d237e47842dec23f26dfab62670799312e29ba27ac92e22488fcb99a236d99d09aa23e09d118870a092047c80b9780967e0454d40e6c2965eacacf8c4bf531405d5213453931357040986bc5966e27af6dc5ade2cb7622a1ea8d750f601fd2c200f2cb596a81425a3e5c1b2c1e708cfe26826fd6247da1c3e4a575a1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from keras import layers
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/aids-data-quest-society/train_ds.csv', index_col=0)
train_df.dropna(inplace=True)
train_df['Acidity'] = train_df['Acidity'].astype(float)
train_df['Quality'] = train_df['Quality'].map({"good": 1, "bad": 0})
plt.rcParams['figure.figsize']=[20,10]
train_df.hist()
plt.show

In [ ]:
plt.rcParams['figure.figsize']=[20,30]
train_df.plot(kind='density',subplots=True,sharex=True)
plt.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_train = scaler.fit_transform(train_df.iloc[:, :-1])
x_train = scaled_train
y_train = train_df['Quality'].values
input_shape = [x_train.shape[1]]

In [ ]:
model = keras.Sequential([
    layers.BatchNormalization(input_shape=input_shape),
    layers.Dense(units=256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(units=128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(units=64, activation='relu'),
    layers.Dense(units=1, activation='sigmoid')
])
model.summary()

In [ ]:
model.summary()
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='binary_accuracy'
)
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=100,
    callbacks=[keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)]
)

In [ ]:
test_df = pd.read_csv('/kaggle/input/aids-data-quest-society/test_ds.csv')


In [ ]:
test_df.dropna(inplace=True)
scaled_test = scaler.transform(test_df.iloc[:, 1:])
submission_predictions = model.predict(scaled_test)
submission_predictions = np.round(submission_predictions).astype(int)
submission_labels = {1: 'good', 0: 'bad'}
submission_predictions = [submission_labels[pred[0]] for pred in submission_predictions]
submission_df = pd.DataFrame({'ID': test_df['ID'], 'Quality': submission_predictions})
submission_df.to_csv('submission.csv', index=False)
submission_df

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error, precision_score, classification_report
y_pred_train = model.predict(x_train)
y_pred_train = np.round(y_pred_train).astype(int)

In [ ]:
acc=accuracy_score(y_train, y_pred_train)
acc

In [ ]:
prec = precision_score(y_train, y_pred_train)
prec

In [ ]:
mae = mean_absolute_error(y_train, y_pred_train)
mae

In [ ]:
report=classification_report(y_train, y_pred_train)
report

In [ ]:
output_file_path = '/kaggle/working/classification_report.csv'
with open(output_file_path, 'w') as f:
    f.write("Accuracy: {}\n".format(acc))
    f.write("Mean Absolute Error: {}".format(mae))
    f.write("Classification Report:\n")
    f.write(report)
output_file_path